# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#!pip install chromadb

Defaulting to user installation because normal site-packages is not writeable


In [24]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [25]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [26]:
llm = OpenAI(temperature=0)

In [29]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [30]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [31]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [32]:
from langchain_community.document_loaders import WebBaseLoader

In [33]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [34]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2130, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

In [35]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [36]:
tools = [
    Tool(
        name="Shakespear Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about one of the most important Shakespeare sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [37]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [41]:
agent.invoke(
    "What did Shakespear say about the beauty of youth?"
)



> Entering new AgentExecutor chain...
 Shakespeare often wrote about the beauty of youth in his sonnets, so I should use the Shakespear Sonnets QA System.
Action: Shakespear Sonnets QA System
Action Input: "What did Shakespeare say about the beauty of youth?"
Observation:  Shakespeare believed that youth's beauty is fleeting and will eventually fade with time. He also believed that beauty should be cherished and passed down through generations.
Thought: This answer seems to align with Shakespeare's overall themes in his sonnets.
Final Answer: Shakespeare believed that youth's beauty is fleeting and should be cherished and passed down through generations.

> Finished chain.


{'input': 'What did Shakespear say about the beauty of youth?',
 'output': "Shakespeare believed that youth's beauty is fleeting and should be cherished and passed down through generations."}

In [42]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff has a larger rule set and does not support custom lint rules, while Flake8 supports plugins and allows for custom rules. Ruff also has the ability to automatically fix its own lint violations.
Thought: This information is helpful, but I still need to consider the specific needs of my project.
Action: Ruff QA System
Action Input: "Which linter is better for a large project with custom lint rules?"
Observation:  Ruff would not be a good choice for a large project with custom lint rules, as it does not support custom or third-party rules. Pylint would be a better option in this case, as it supports plugins and allows for the creation of custom lint rules.
Thought: Pylint seems like a good option, but I want to make sure I'm not missing any i

{'input': 'Why use ruff over flake8?',
 'output': 'It ultimately depends on the specific needs and preferences of the project, but Pylint may be a better choice for a large project with custom lint rules, while Ruff may be a better choice for a project that values a larger rule set and the ability to automatically fix lint violations.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [43]:
tools = [
    Tool(
        name="Shakespear Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about one of the most important Shakespeare sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [44]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [45]:
agent.invoke(
    "What did say about the beauty of youth?"
)



> Entering new AgentExecutor chain...
 I should use the Shakespear Sonnets QA System to answer this question.
Action: Shakespear Sonnets QA System
Action Input: "What did Shakespeare say about the beauty of youth?"
Observation:  Shakespeare believed that youth's beauty is fleeting and will eventually fade with time. He also believed that beauty should be cherished and passed down through generations.


> Finished chain.


{'input': 'What did say about the beauty of youth?',
 'output': " Shakespeare believed that youth's beauty is fleeting and will eventually fade with time. He also believed that beauty should be cherished and passed down through generations."}

In [46]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff has a larger rule set and does not support custom lint rules, while Flake8 supports plugins and allows for custom rules. Ruff also has the ability to automatically fix its own lint violations.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff has a larger rule set and does not support custom lint rules, while Flake8 supports plugins and allows for custom rules. Ruff also has the ability to automatically fix its own lint violations.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [47]:
tools = [
    Tool(
        name="Shakespear Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about one of the most important Shakespeare sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [48]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [49]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did shakespear talk about the youth?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the Shakespear Sonnets QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  I don't know.
Thought: I should try rephrasing the question.
Action: Ruff QA System
Action Input: What is the name of the tool that ruff uses to run over Jupyter Notebooks?
Observation:  I don't know.
Thought: I should try asking about ruff's features.
Action: Ruff QA System
Action Input: What are some features of ruff?
Observation:  Ruff has a linter that is compatible with Black and Flake8, and a formatter that is designed to be a drop-in replacement for Black. It also has the ability to implement popular Flake8 plugins and code quality tools.
Thought: I should now use the Shakespear Sonnets QA System to answer the second question.
Action: Shakespear Sonnets QA System
Action Input: Did shakespear talk about the y

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did shakespear talk about the youth?',
 'output': 'Yes, Shakespeare did talk about the youth in his sonnets.'}